In [12]:
import agate

In [13]:
exonerations = agate.Table.from_csv('exonerations-20150828.csv')

In [14]:
print(exonerations)

|----------------------+---------------|
|  column_names        | column_types  |
|----------------------+---------------|
|  last_name           | Text          |
|  first_name          | Text          |
|  age                 | Number        |
|  race                | Text          |
|  state               | Text          |
|  tags                | Text          |
|  crime               | Text          |
|  sentence            | Text          |
|  convicted           | Number        |
|  exonerated          | Number        |
|  dna                 | Boolean       |
|  dna_essential       | Text          |
|  mistake_witness     | Boolean       |
|  false_confession    | Boolean       |
|  perjury             | Boolean       |
|  false_evidence      | Boolean       |
|  official_misconduct | Boolean       |
|  inadequate_defense  | Boolean       |
|----------------------+---------------|



In [15]:
print exonerations.columns[0]

<agate.Column: (u'Abbitt', u'Abdal', u'Abernathy', u'Acero', u'Adams', ...)>


In [16]:
first_row=exonerations.rows[0]
print first_row['dna']


True


In [17]:
exonerations = agate.Table.from_csv('exonerations-20150828.csv', columns, row_names=lambda r: '%(last_name)s, %(first_name)s' % (r))
exonerations.rows[0]

NameError: name 'columns' is not defined

In [ ]:
column = exonerations.columns['crime']
print column[0]

In [ ]:
row = exonerations.rows[0]
print row

In [ ]:
row = exonerations.rows[0]
print row['last_name']


In [ ]:
column = exonerations.columns['crime']
print column[0]
print column['Abbitt, Joseph Lamont']

In [ ]:
for row in exonerations.rows:
    print(row['last_name'])

In [ ]:
num_false_confessions = exonerations.aggregate(agate.Count('false_confession', True))

print(num_false_confessions)

In [ ]:
median_age = exonerations.aggregate(agate.Median('age'))

print(median_age)

In [ ]:
num_without_age = exonerations.aggregate(agate.Count('age', None))

print(num_without_age)

In [ ]:
with_age = exonerations.where(lambda row: row['age'] is not None)


In [ ]:
old = len(exonerations.rows)
new = len(with_age.rows)

print(old - new)

In [ ]:
median_age = with_age.aggregate(agate.Median('age'))

print(median_age)

In [ ]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

median_years = with_years_in_prison.aggregate(agate.Median('years_in_prison'))

print(median_years)

In [ ]:
full_names = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row))
])

In [ ]:
print full_names

In [ ]:
print exonerations.columns['full_name']

In [ ]:
for row in exonerations.rows:
    print(row['full_name'])

In [ ]:
with_computations = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row)),
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

In [ ]:
sorted_by_age = exonerations.order_by('age')

In [ ]:
youngest_ten = sorted_by_age.limit(10)

In [ ]:
youngest_ten.print_table(max_columns=7)

In [ ]:
binned_ages = exonerations.bins('age', 10, 0, 100)
binned_ages.print_bars('age', 'Count', width=80)

In [ ]:
by_state = exonerations.group_by('state')
state_totals = by_state.aggregate([
    ('count', agate.Count())
])

sorted_totals = state_totals.order_by('count', reverse=True)

sorted_totals.print_table(max_rows=5)

In [ ]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

state_totals = with_years_in_prison.group_by('state')

medians = state_totals.aggregate([
    ('count', agate.Count()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

sorted_medians = medians.order_by('median_years_in_prison', reverse=True)

sorted_medians.print_table(max_rows=5)

In [ ]:
# Filters rows without age data
only_with_age = with_years_in_prison.where(
    lambda r: r['age'] is not None
)

# Group by race
race_groups = only_with_age.group_by('race')

# Sub-group by age cohorts (20s, 30s, etc.)
race_and_age_groups = race_groups.group_by(
    lambda r: '%i0s' % (r['age'] // 10),
    key_name='age_group'
)

# Aggregate medians for each group
medians = race_and_age_groups.aggregate([
    ('count', agate.Count()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

# Sort the results
sorted_groups = medians.order_by('median_years_in_prison', reverse=True)

# Print out the results
sorted_groups.print_table(max_rows=10)

In [ ]:
exonerations = agate.Table.from_csv('exonerations-20150828.csv')
mo = exonerations.aggregate([agate.Count('state', 'mo')])
print mo

In [ ]:
exonerations = agate.Table.from_csv('exonerations-20150828.csv')
mo = exonerations.aggregate(agate.Count('state', 'MO'))
print mo